# **Task 1**

In [31]:
from pyspark.sql import *
from pyspark.sql.functions import split
import pandas as pd

spark = SparkSession.builder.getOrCreate()
RatingsDF = spark.read.text("ratings.txt")
RatingsDF = RatingsDF.withColumn('UserID', split(RatingsDF['value'], ' ').getItem(0))
RatingsDF = RatingsDF.withColumn('ItemID', split(RatingsDF['value'], ' ').getItem(1))
RatingsDF = RatingsDF.withColumn('Rating', split(RatingsDF['value'], ' ').getItem(2))
RatingsDF = RatingsDF.drop('value')
RatingsDF.dropna()

35497
35497


https://saturncloud.io/blog/how-to-split-a-column-into-multiple-columns-in-pyspark-without-using-pandas/
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumn.html